In [2]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("mini-contest/dataset-agent.csv")

# Display the first few rows of the DataFrame
print(df.head(1))



   id                                          project_a  \
0   2  https://github.com/prettier-solidity/prettier-...   

                                    project_b  weight_a  weight_b  \
0  https://github.com/nomicfoundation/hardhat  0.101669  0.898331   

   agent_weight_a  agent_weight_b agent_trace_url  
0        0.317076        0.682924             NaN  


In [3]:
def flatten_dict(d, parent_key="", sep="_"):
    """Flatten a nested dictionary into a single level dictionary with concatenated keys."""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [4]:
# force a fresh import of the module, otherwise you need to restart the ipython kernel whenever you make changes to the deepfunding.py file
import importlib
import deepfunding
importlib.reload(deepfunding)

from deepfunding import run_comparison
from tools.data_collection import fetch_oso_metrics

# Initialize agent weight columns if they don't exist
if 'agent_weight_a' not in df.columns:
    df['agent_weight_a'] = None
if 'agent_weight_b' not in df.columns:
    df['agent_weight_b'] = None
if 'agent_trace_url' not in df.columns:
    df['agent_trace_url'] = None

for i in range(len(df)):
    if pd.notnull(df.at[i, 'agent_weight_a']) and pd.notnull(df.at[i, 'agent_weight_b']):
        continue

    repo_a_url = df.iloc[i]["project_a"]
    repo_b_url = df.iloc[i]["project_b"]
    print(f"{repo_a_url} VS {repo_b_url}")
    repo_a = { "url": repo_a_url }
    repo_b = { "url": repo_b_url }

    agent_result = run_comparison(repo_a, repo_b)
    flat_result = flatten_dict(agent_result)

    # Write all flattened keys to DataFrame
    for key, value in flat_result.items():
        # Create column if it doesn't exist
        if key not in df.columns:
            df[key] = None

        df.at[i, key] = value

    # Keep the original weight assignments for compatibility
    df.at[i, "agent_weight_a"] = agent_result["weights"][repo_a_url]
    df.at[i, "agent_weight_b"] = agent_result["weights"][repo_b_url]
    df.at[i, "agent_trace_url"] = agent_result["trace_url"]

    print(f"Agent: {df.iloc[i]['agent_weight_a']} {df.iloc[i]['agent_weight_b']}")
    print(f"Human: {df.iloc[i]['weight_a']} {df.iloc[i]['weight_b']}")
    print(f"Check details here: {df.iloc[i]['agent_trace_url']}")
    df.to_csv("mini-contest/dataset-agent.csv", index=False)

https://github.com/walletconnect/walletconnect-monorepo VS https://github.com/pnpm/cmd-shim
funding_strategist Result: weight_a=0.4 weight_b=0.6 confidence=0.9 reasoning='Repo A has a strong community with a high contributor count and significant activity (commit count and active developers), indicating ongoing support and development. However, it faces challenges with a high number of open issues, suggesting potential sustainability risks. Repo B, despite having a larger star count, shows increasing funding momentum and activity with new contributors and consistent commits, which may indicate a growing project. Therefore, I recommend a balanced approach, favoring Repo B for its growth and funding trends while still supporting Repo A for its established utility.' metrics_used=['activeDeveloperCount6Months', 'commitCount6Months', 'contributorCount', 'starCount', 'total_funding_received_usd', 'total_funders_count']
project_analyzer Result: weight_a=0.55 weight_b=0.45 confidence=0.85 reas

KeyError: 'weights'